In [407]:
import requests
import os
import json
from pprint import pprint

# os.environ['BEARER_TOKEN'] = 'WRITE YOUR TOKEN HERE'
bearer_token = os.environ.get('BEARER_TOKEN')

def create_url(_ids):
#     tweet_fields = "tweet.fields=lang,author_id,attachments,context_annotations,conversation_id,created_at,entities,geo,id,public_metrics,withheld"
#     expansion = "expansions=author_id"
#     author_fields = "user.fields=id,name,username,created_at,description,entities,location,public_metrics,protected,verified,withheld"
    ids = "ids={}".format(_ids)
    tweet_fields = "tweet.fields=lang,created_at,geo,id,public_metrics,withheld"
    expansion = "expansions=author_id"
    user_fields = "user.fields=created_at,description,public_metrics,protected,verified,withheld"
    
    url = "https://api.twitter.com/2/tweets?{}&{}&{}&{}".format(ids, tweet_fields, expansion, user_fields)
    return url


def get_tweets(_ids):
    url = create_url(_ids)
    payload={}
    headers = {
      'Authorization': 'Bearer {}'.format(bearer_token),
      'Cookie': 'guest_id=v1%3A164999832374703747; guest_id_ads=v1%3A164999832374703747; guest_id_marketing=v1%3A164999832374703747; personalization_id="v1_+F68isE/iukb7yr8y66bOw=="'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    return response.json()


In [2]:
import pandas as pd

def read_file(file_name):
    '''
    This function will read the file containing tweet ids sperating by lines (in groups)
    '''
    all_ids = pd.read_table(file_name,sep='\n',header=None)
    return all_ids

In [3]:
def separate_ids(ids):
    '''
    This function will split the sperate ids into a list
    '''
    seperated_ids = ids.split(",")
    return seperated_ids

In [51]:
def get_tweet_groups(groups_ids):
    '''
    This function will get tweets information using Twitter API in groups with indexing
    params groups_ids: all ids in lines (groups) retrieved from the orginal file 
    '''
    tweet_groups = {}
    no_access_first_tweets = []
    for group_index in range(len(groups_ids)):
        group_separated_ids = separate_ids(groups_ids[0][group_index])
        group_dict = {}
        
        # first check if the first tweet is valid to be accessed
        # if no access, only 'errors' will be returned
        first_tweet = get_tweets(group_separated_ids[0])
        if len(first_tweet) == 1:
            no_access_first_tweets.append(group_index)
            continue
        else:
            # check if the group contains more than 100 ids 
            # since multiple tweets lookup has 100 maxmimum
            if len(group_separated_ids) > 100:
                index = 0
                partial_group = {}

                # partition groups with more than 100 ids into maximum 100 each group
                for i in range(len(group_separated_ids)//100 + 1):
                    partial_group_ids = ','.join(group_separated_ids[index:index+100])
                    partial_group = get_tweets(partial_group_ids)
                    # initialise the dict with the information from the first partial group
                    if len(group_dict) == 0 and len(partial_group) != 1:
                        group_dict = partial_group
                    # combining partial groups into a whole group
                    else:
                        # there is any tweet valid to be accessed in the partial group
                        if len(partial_group) != 1:
                            group_dict['data'].extend(partial_group['data'])
                            group_dict['includes']['users'].extend(partial_group['includes']['users'])

                    index += 100

            else:
                full_group_ids = ','.join(group_separated_ids)
                group_dict = get_tweets(full_group_ids)

            tweet_groups[group_index] = group_dict
    return tweet_groups, no_access_first_tweets

In [85]:
def get_tweet_groups_start_end(groups_ids, start, end):
    '''
    This function will get tweets information using Twitter API in groups with indexing
    params groups_ids: all ids in lines (groups) retrieved from the orginal file 
    '''
    tweet_groups = {}
    no_access_first_tweets = []
    for group_index in range(start, end):
        group_separated_ids = separate_ids(groups_ids[0][group_index])
        group_dict = {}
        
        # first check if the first tweet is valid to be accessed
        # if no access, only 'errors' will be returned
        first_tweet = get_tweets(group_separated_ids[0])
        if len(first_tweet) == 1:
            no_access_first_tweets.append(group_index)
            continue
        else:
            # check if the group contains more than 100 ids 
            # since multiple tweets lookup has 100 maxmimum
            if len(group_separated_ids) > 100:
                index = 0
                partial_group = {}

                # partition groups with more than 100 ids into maximum 100 each group
                for i in range(len(group_separated_ids)//100 + 1):
                    if len(group_separated_ids) % 100 == 0 and i == len(group_separated_ids)//100:
                        continue
                        
                    partial_group_ids = ','.join(group_separated_ids[index:index+100])
                    partial_group = get_tweets(partial_group_ids)

                    # initialise the dict with the information from the first partial group
                    if len(group_dict) == 0 and len(partial_group) != 1:
                        group_dict = partial_group
                    # combining partial groups into a whole group
                    else:
                        # there is any tweet valid to be accessed in the partial group
                        if len(partial_group) != 1:
                            group_dict['data'].extend(partial_group['data'])
                            group_dict['includes']['users'].extend(partial_group['includes']['users'])

                    index += 100

            else:
                full_group_ids = ','.join(group_separated_ids)
                group_dict = get_tweets(full_group_ids)

            tweet_groups[group_index] = group_dict
    return tweet_groups, no_access_first_tweets

In [299]:
train_ids = read_file('../../project-data/train.data.txt')
train_ids

,0
0,"1250219300389974016,1250219116993974272,125021..."
1,"554886875303780352,554894001946759168,55495964..."
2,"1237901309011021825,1237901311439450112,123986..."
3,"524958128392376320,524961934064754688,52495902..."
4,"1239295488677085185,1239562248990806016"
...,...
1890,"671181758692507648,671200376843067392,67118426..."
1891,"672513234419638273,672513853645717504,67258916..."
1892,"553508098825261056,553511652587360256,55350954..."
1893,"1249479605582327808,1249392473073487873,124953..."


In [285]:
dev_ids = read_file('../../project-data/dev.data.txt')
dev_ids

,0
0,"1249004694950817796,1249011200068730880"
1,"1267552274819227649,1270394169836568576,127050..."
2,"1235238334722699265,1235234904281165825,123523..."
3,"1248746792914546688,1248775858120097792"
4,"523820806917603328,523943560589361152,52402705..."
...,...
627,"1249582429565829120,1249582412004306944,124958..."
628,"524881688825167872,524919134715252736,52493671..."
629,"1240908749256232960,1240908751063965696,124090..."
630,"661102820976930816,661643543551848448,66115142..."


### Devset

In [34]:
# tweets_dev1, no_access_first_tweets_dev1 = get_tweet_groups_start_end(dev_ids, 0, 100)

In [161]:
# tweets_dev2, no_access_first_tweets_dev2 = get_tweet_groups_start_end(dev_ids, 100, 200)

In [275]:
# tweets_dev3, no_access_first_tweets_dev3 = get_tweet_groups_start_end(dev_ids, 200, 300)

In [571]:
# tweets_dev4, no_access_first_tweets_dev4 = get_tweet_groups_start_end(dev_ids, 300, 400)

In [286]:
# tweets_dev5, no_access_first_tweets_dev5 = get_tweet_groups_start_end(dev_ids, 400, 500)

In [293]:
# tweets_dev6, no_access_first_tweets_dev6 = get_tweet_groups_start_end(dev_ids, 500, 632)

In [619]:
with open("data/dev_data.json",'r',encoding='utf-8') as f:
    tweet_dev = json.load(f)
    
with open("data/no_access_first_tweets_dev.json",'r',encoding='utf-8') as f:
    no_access_first_tweets_dev = json.load(f)

In [589]:
# tweet_dev.update(tweets_dev4)

# no_access_first_tweets_dev.extend(no_access_first_tweets_dev4)

In [613]:
with open("data/dev_data.json", "w",encoding='utf-8') as outfile:
    json.dump(tweet_dev, outfile)
        
with open("data/no_access_first_tweets_dev.json", "w",encoding='utf-8') as outfile:
    json.dump(no_access_first_tweets_dev, outfile)

### Trainset

In [161]:
# tweets_train1, no_access_first_tweets1 = get_tweet_groups_start_end(train_ids, 0, 100)

In [181]:
# tweets_train2, no_access_first_tweets2 = get_tweet_groups_start_end(train_ids, 100, 200)

In [183]:
# tweets_train3, no_access_first_tweets3 = get_tweet_groups_start_end(train_ids, 200, 300)

In [189]:
# tweets_train4, no_access_first_tweets4 = get_tweet_groups_start_end(train_ids, 300, 400)

In [86]:
# tweets_train5, no_access_first_tweets5 = get_tweet_groups_start_end(train_ids, 400, 500)

In [124]:
# tweets_train6, no_access_first_tweets6 = get_tweet_groups_start_end(train_ids, 500, 600)

In [136]:
# tweets_train7, no_access_first_tweets7 = get_tweet_groups_start_end(train_ids, 600, 700)

In [190]:
# tweets_train8, no_access_first_tweets8 = get_tweet_groups_start_end(train_ids, 700, 800)

In [ ]:
# tweets_train9, no_access_first_tweets9 = get_tweet_groups_start_end(train_ids, 800, 900)

In [198]:
# tweets_train10, no_access_first_tweets10 = get_tweet_groups_start_end(train_ids, 900, 1000)

In [504]:
# tweets_train11, no_access_first_tweets11 = get_tweet_groups_start_end(train_ids, 1000, 1100)

In [506]:
# tweets_train12, no_access_first_tweets12 = get_tweet_groups_start_end(train_ids, 1100, 1200)

In [213]:
# tweets_train13, no_access_first_tweets13 = get_tweet_groups_start_end(train_ids, 1200, 1300)

In [222]:
# tweets_train14, no_access_first_tweets14 = get_tweet_groups_start_end(train_ids, 1300, 1400)

In [232]:
# tweets_train15, no_access_first_tweets15 = get_tweet_groups_start_end(train_ids, 1400, 1500)

In [242]:
# tweets_train16, no_access_first_tweets16 = get_tweet_groups_start_end(train_ids, 1500, 1600)

In [250]:
# tweets_train17, no_access_first_tweets17 = get_tweet_groups_start_end(train_ids, 1600, 1700)

In [263]:
# tweets_train18, no_access_first_tweets18 = get_tweet_groups_start_end(train_ids, 1700, 1800)

In [413]:
# tweets_train19, no_access_first_tweets19 = get_tweet_groups_start_end(train_ids, 1800, 1895)

In [265]:
len(tweets_train18)

175

In [267]:
len(no_access_first_tweets18)

20

In [630]:
tweet_train['1200']['includes']

{'users': [{'public_metrics': {'followers_count': 367901,
    'following_count': 5211,
    'tweet_count': 34480,
    'listed_count': 5483},
   'username': 'rcallimachi',
   'name': 'Rukmini Callimachi',
   'created_at': '2009-04-12T18:58:23.000Z',
   'protected': False,
   'description': 'New York Times journalist currently on book leave. NBC analyst. Previously, 7 years covering ISIS & al-Qaeda, 7 years in Africa. Ex-AP bureau chief. Ex-refugee.',
   'verified': True,
   'id': '30699754'},
  {'public_metrics': {'followers_count': 2419,
    'following_count': 1679,
    'tweet_count': 86616,
    'listed_count': 20},
   'username': 'GleamingRazor',
   'name': 'GleamingRazor',
   'created_at': '2013-08-13T20:31:46.000Z',
   'protected': False,
   'description': 'RT/LIKE not necessarily an endorsement. I love swimming, football, gym, reading, writing and annoying people on Twitter. Muslim.',
   'verified': False,
   'id': '1668697268'}]}

In [622]:
with open("data/train_data.json",'r',encoding='utf-8') as f:
    tweet_train = json.load(f)
    
with open("data/no_access_first_tweets_train.json",'r',encoding='utf-8') as f:
    no_access_first_tweets_train = json.load(f)

In [553]:
# tweet_train.update(tweets_train12)

# no_access_first_tweets_train.extend(no_access_first_tweets12)

In [569]:
with open("data/train_data.json", "w",encoding='utf-8') as outfile:
    json.dump(tweet_train, outfile)
        
with open("data/no_access_first_tweets_train.json", "w",encoding='utf-8') as outfile:
    json.dump(no_access_first_tweets_train, outfile)